# IR Lab SoSe 2024: Vergleichendes Retrieval System

Dieses Jupyter Notebook implementiert und vergleicht mehrere Retrieval-Ansätze:
1. Nur BM25
2. BM25 + TF-IDF
3. BM25 + Gewichtetes TF-IDF (mit Feldgewichtung)

Wir verwenden ein Korpus wissenschaftlicher Arbeiten (Titel + Abstracts) aus den Bereichen Information Retrieval und Natural Language Processing.

### Schritt 1: Bibliotheken importieren und Setup

In [1]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from pyterrier.pipelines import *
import pandas as pd

ensure_pyterrier_is_loaded()
tira = Client()

# Das Dataset und der Index
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


### Schritt 2: Retrieval Pipelines definieren

In [2]:
# 1. Nur BM25
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

# 2. BM25 + TF-IDF
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25_tfidf = bm25 >> tfidf

# 3. BM25 + Gewichtetes TF-IDF
fields = ["title", "abstract"]
weights = [2.0, 1.0]  # Titel hat doppeltes Gewicht des Abstracts
weighted_tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", controls={"c": 1.0}, fields=fields)
for field, weight in zip(fields, weights):
    weighted_tfidf.controls[f"w.{field}"] = weight
bm25_weighted_tfidf = bm25 >> weighted_tfidf

### Schritt 3: Retrieval durchführen

In [3]:
topics = pt_dataset.get_topics('text')
print('Die ersten drei Topics:')
print(topics.head(3))

print('\nFühre Retrieval für alle Ansätze durch...')
run_bm25 = bm25(topics)
run_bm25_tfidf = bm25_tfidf(topics)
run_bm25_weighted_tfidf = bm25_weighted_tfidf(topics)
print('Retrieval abgeschlossen.')

Die ersten drei Topics:
  qid                                     query
0   1  retrieval system improving effectiveness
1   2  machine learning language identification
2   3             social media detect self harm

Führe Retrieval für alle Ansätze durch...


BR(TF_IDF): 100%|██████████| 68/68 [00:01<00:00, 62.02q/s]


Retrieval abgeschlossen.


### Schritt 4: Ergebnisse vergleichen

In [4]:
def compare_top_k(runs, k=10):
    results = {}
    for name, run in runs.items():
        results[name] = run.groupby('qid').head(k)
    
    comparison = pd.DataFrame()
    for name, result in results.items():
        comparison[f'{name}_docno'] = result['docno']
        comparison[f'{name}_score'] = result['score']
    
    return comparison

runs = {
    'BM25': run_bm25,
    'BM25+TF-IDF': run_bm25_tfidf,
    'BM25+Weighted-TF-IDF': run_bm25_weighted_tfidf
}

comparison = compare_top_k(runs)
print(f'Vergleich der Top 10 Ergebnisse für die erste Anfrage (qid=1):')
print(comparison[comparison.index.get_level_values('qid') == '1'])

# Berechnen Sie den Overlap zwischen den Top-K Ergebnissen
def calculate_overlap(runs, k=10):
    overlaps = {}
    for name1, run1 in runs.items():
        for name2, run2 in runs.items():
            if name1 < name2:  # Vermeiden Sie doppelte Vergleiche
                overlap = len(set(run1.groupby('qid').head(k)['docno']) & 
                              set(run2.groupby('qid').head(k)['docno']))
                overlaps[f'{name1} vs {name2}'] = overlap
    return overlaps

overlaps = calculate_overlap(runs)
print(f'\nÜberlappung der Top 10 Ergebnisse zwischen den Ansätzen:')
for comparison, overlap in overlaps.items():
    print(f'{comparison}: {overlap} von 10')

Vergleich der Top 10 Ergebnisse für die erste Anfrage (qid=1):


KeyError: 'Requested level (qid) does not match index name (None)'

### Schritt 5: Run-Dateien persistieren

In [ ]:
persist_and_normalize_run(run_bm25, system_name='bm25-baseline', default_output='../runs')
persist_and_normalize_run(run_bm25_tfidf, system_name='bm25-tfidf-combined', default_output='../runs')
persist_and_normalize_run(run_bm25_weighted_tfidf, system_name='bm25-weighted-tfidf-combined', default_output='../runs')

### Schritt 6: Analyse und Interpretation

1. Betrachten Sie die Top-10-Ergebnisse für verschiedene Anfragen. Wie unterscheiden sich die Ergebnisse zwischen den Ansätzen?

2. Analysieren Sie die Überlappung der Ergebnisse. Ein hoher Überlappungsgrad deutet darauf hin, dass die Ansätze ähnliche Ergebnisse liefern, während ein niedriger Grad auf unterschiedliche Stärken der Ansätze hinweisen könnte.

3. Untersuchen Sie spezifische Fälle, in denen die gewichtete TF-IDF-Methode andere Ergebnisse liefert als die anderen Methoden. Sind diese Unterschiede auf die Titelgewichtung zurückzuführen?

4. Betrachten Sie die Scores der verschiedenen Methoden. Wie unterscheiden sie sich in ihrer Verteilung und Größenordnung?

5. Für eine gründlichere Evaluation wären Relevanzurteile erforderlich, um Metriken wie MAP oder NDCG zu berechnen. In Ermangelung solcher Urteile könnten Sie eine manuelle Stichprobenprüfung der Ergebnisse für einige ausgewählte Anfragen durchführen.

6. Überlegen Sie, wie Sie die Ansätze weiter verbessern könnten. Mögliche Richtungen umfassen:
   - Experimentieren mit verschiedenen Feldgewichten
   - Einbeziehung zusätzlicher Felder (z.B. Autorennamen, Veröffentlichungsjahr)
   - Implementierung von Query Expansion oder Pseudo-Relevanz-Feedback
   - Verwendung von neueren Ranking-Modellen wie BERT oder andere Transformer-basierte Modelle

Denken Sie daran, dass die 'beste' Methode oft von der spezifischen Aufgabe und dem Datensatz abhängt. Eine gründliche Evaluation und Analyse ist entscheidend, um zu verstehen, welcher Ansatz für Ihre spezifischen Bedürfnisse am besten geeignet ist.